In [2]:
import pandas as pd

In [3]:
movie = pd.read_csv(r'C:\Users\hp\Downloads\movie_data\movies.csv')

In [ ]:
movie.shape

(9742, 3)

In [4]:
rating = pd.read_csv(r'C:\Users\hp\Downloads\movie_data\ratings.csv')
rating.shape

(100836, 4)

In [ ]:
rating.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
# Get the unique userId values
unique_user_ids = rating['userId'].unique()

# Count the number of unique userIds
num_unique_user_ids = len(unique_user_ids)

print(f'Number of unique userIds: {num_unique_user_ids}')

Number of unique userIds: 610


In [7]:
grouped_ratings = rating.groupby('movieId')['rating'].agg(['count', 'mean']).reset_index()

In [8]:
# Merge with movies data to get movie titles
merged_df = pd.merge(grouped_ratings, movie, on='movieId')


In [9]:
# Filter movies with more than 50 ratings
filtered_df = merged_df[merged_df['count'] > 50]
filtered_df.head()

,movieId,count,mean,title,genres
0,1,215,3.920930,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,110,3.431818,Jumanji (1995),Adventure|Children|Fantasy
2,3,52,3.259615,Grumpier Old Men (1995),Comedy|Romance
5,6,102,3.946078,Heat (1995),Action|Crime|Thriller
6,7,54,3.185185,Sabrina (1995),Comedy|Romance


In [10]:
# Find the movie with the highest average rating
most_popular_movie = filtered_df.loc[filtered_df['mean'].idxmax()]

# Print details of the most popular movie
print('Most popular movie based on average user ratings:')
print(f"Title: {most_popular_movie['title']}")
print(f"Average Rating: {most_popular_movie['mean']:.2f}")
print(f"Number of Ratings: {most_popular_movie['count']}")


Most popular movie based on average user ratings:
Title: Shawshank Redemption, The (1994)
Average Rating: 4.43
Number of Ratings: 317


In [11]:
# Sort by the number of ratings in descending order
top_movies = filtered_df.sort_values(by='count', ascending=False).head(5)

# Print the top 5 movies based on number of ratings
print('Top 5 movies based on number of user ratings:')
for index, row in top_movies.iterrows():
    print(f"Title: {row['title']}, Number of Ratings: {row['count']}")


Top 5 movies based on number of user ratings:
Title: Forrest Gump (1994), Number of Ratings: 329
Title: Shawshank Redemption, The (1994), Number of Ratings: 317
Title: Pulp Fiction (1994), Number of Ratings: 307
Title: Silence of the Lambs, The (1991), Number of Ratings: 279
Title: Matrix, The (1999), Number of Ratings: 278


In [12]:
# Filter for Sci-Fi movies
sci_fi_movies = filtered_df[filtered_df['genres'].str.contains('Sci-Fi', case=False, na=False)]


In [13]:
# Sort Sci-Fi movies by number of ratings in descending order
sorted_sci_fi_movies = sci_fi_movies.sort_values(by='count', ascending=False)

# Get the third most popular Sci-Fi movie
third_most_popular_sci_fi = sorted_sci_fi_movies.iloc[2]  # Index 2 for the third movie

# Print the details
print('Third most popular Sci-Fi movie based on number of user ratings:')
print(f"Title: {third_most_popular_sci_fi['title']}")
print(f"Number of Ratings: {third_most_popular_sci_fi['count']}")


Third most popular Sci-Fi movie based on number of user ratings:
Title: Jurassic Park (1993)
Number of Ratings: 238


In [16]:
links_df = pd.read_csv(r'C:\Users\hp\Downloads\movie_data\links.csv')

In [23]:
# Merge with links to get IMDb IDs
movies_with_links = pd.merge(filtered_df, links_df, on='movieId')


In [30]:
import requests
from bs4 import BeautifulSoup

def fetch_imdb_reviews(imdb_id):
    url = f'https://www.imdb.com/title/{imdb_id}/reviews'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Parse reviews - this will depend on IMDb's HTML structure
    reviews = []
    for review in soup.find_all('div', class_='text show-more__control'):
        reviews.append(review.get_text())

    return reviews
# Example to scrape reviews for a single movie
imdb_id = movies_with_links.iloc[0]['imdbId']
reviews = fetch_imdb_reviews(imdb_id)
print(f'Reviews for IMDb ID {imdb_id}:')
for review in reviews:
    print(review)



ConnectionError: HTTPSConnectionPool(host='www.imdb.com', port=443): Max retries exceeded with url: /title/114709/reviews (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000021F313C0EB0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [28]:
import requests
from bs4 import BeautifulSoup

def fetch_imdb_rating(imdb_id):
    url = f'https://www.imdb.com/title/{imdb_id}/'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Parse IMDb rating
    rating_tag = soup.find('span', itemprop='ratingValue')
    if rating_tag:
        return float(rating_tag.get_text())
    return None

# Example to get IMDb rating for a single movie
imdb_id = 'tt0111161'  # Replace with actual IMDb ID
rating = fetch_imdb_rating(imdb_id)
print(f'IMDb rating for IMDb ID {imdb_id}: {rating}')


ConnectionError: HTTPSConnectionPool(host='www.imdb.com', port=443): Max retries exceeded with url: /title/tt0111161/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000021F31324220>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [27]:
with open('movie_reviews.txt', 'w') as file:
    for imdb_id in movies_with_links['imdbId']:
        reviews = fetch_imdb_reviews(imdb_id)
        file.write(f'IMDb ID: {imdb_id}\n')
        for review in reviews:
            file.write(f'{review}\n')
        file.write('\n' + '-'*50 + '\n')


ConnectionError: HTTPSConnectionPool(host='www.imdb.com', port=443): Max retries exceeded with url: /title/114709/reviews (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000021F31182340>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [ ]:
ratings = []
for imdb_id in movies_with_links['imdbId']:
    rating = fetch_imdb_rating(imdb_id)
    if rating:
        ratings.append((imdb_id, rating))


In [21]:
# Convert list of ratings to DataFrame
ratings_df = pd.DataFrame(ratings, columns=['imdbId', 'rating'])

# Merge with the movies_with_links DataFrame to get movieId
merged_ratings_df = pd.merge(ratings_df, movies_with_links, on='imdbId')

# Find the movieId with the highest rating
highest_rated_movie = merged_ratings_df.loc[merged_ratings_df['rating'].idxmax()]
highest_movie_id = highest_rated_movie['movieId']

print(f'MovieId of the movie with the highest IMDb rating: {highest_movie_id}')


NameError: name 'ratings' is not defined